# Step 1: prepare data from JH about confirmed cases

We need to get the approximate growth rate of the number of confirmed case for each country. This returns a dataset of all countries with more than 250 cases to this date and their respective growth rate. We will use this as training data

In [111]:
#STEP1: prepare data from JH about confirmed cases
#import data
import pandas as pd
import numpy as np
URL1 = "https://raw.githubusercontent.com/AlexOktay/covid19/master/time_series_covid19_confirmed_global.csv"
country_data = pd.read_csv(URL1)

#Drop LAT and LONG columns, rename "Country/Region"
country_data = country_data.drop(["Lat", "Long"], axis = 1)
country_data.rename(columns={"Country/Region":"Countries"}, inplace=True)

#Agregate data from provinces to countries
dates = list(country_data.columns)
country_data = country_data[dates].groupby(country_data["Countries"]).sum()

# we want to see a daily growth rate for number of cases (assumption: exponential)
# We first need to see when the epidemic started spreading in each country
spread_start = pd.DataFrame(columns=["Country", "Start"])
for index, row in country_data.iterrows():
  for date in country_data:
    if country_data.loc[index, date] > 0:
      spread_start = spread_start.append({"Country":index, "Start":date}, ignore_index=True)
      break  
    else:
      pass

# exponential growth is based on equation y(t) = a__e^rt for a the initial value,
# y(t) the current value, r the growth rate and t the number of days since start of pandemic
# if we resolve for r: r = ln(y(t)/a) / t 
# for simplification purposes, we estimate a=1 (there is only one initial patient)
def rate(y, t):
  "Calculate the exponantial rate of growth"
  growth = np.log(y) / t
  return growth

# Create a new column with growth rate for each country
country_data.insert(0, "Growth_rate",0.00)
last_update = country_data.columns[-1]
for index, row in country_data.iterrows():
  temp = 0
  start_date = spread_start.iloc[temp,1]
  t = len(country_data.columns) - (country_data.columns.get_loc(start_date)+1)
  y = country_data.loc[index, last_update]
  country_data.at[index, 'Growth_rate'] = rate(y,t)
  temp = temp+1

# for training, we will only use countries whcih already have a heavy number
# of confirmed cases (here we set the threshold at >250). We will try to predict
# growth rate for countries under 250 
country_data_copy = country_data #useful in step 4
for index, row in country_data.iterrows():
  if country_data.loc[index, last_update] < 250:
    country_data = country_data.drop(index)
  else:
    pass

# finaly, drop columns with the exact evolution of confirmed cases: we will be 
#working only on the growth_rate to try to predict it
country_data.drop(country_data.iloc[:, 1:], axis=1, inplace=True)

country_data

,Growth_rate
Countries,
Algeria,0.187929
Andorra,0.174602
Argentina,0.199326
Armenia,0.181127
Australia,0.251654
...,...
Turkey,0.270246
US,0.360292
Ukraine,0.179268


# Step 2: Set the economic features
We will use economic data to try to predict the actual growth rate of the epidemic in countries. This will be the features used in our model. This returns a table where each row = one heavily infected country with their respective economic features (columns)

We uses two databases for this:
- general_data for basic features about population (population, density...)
- bank_data for specific economic indicators (from World Bank)

## Step 2.1 First data source: general_data

In [112]:
# import the data
URL2 = "https://raw.githubusercontent.com/AlexOktay/covid19/master/countries%20of%20the%20world.csv"
general_data = pd.read_csv(URL2)

#Remove space at the end of the country name in general data, and edit names to match country_data
for index, row in general_data.iterrows():  
  general_data.iloc[index, 0] = general_data.iloc[index, 0].rstrip()
general_data.Country.replace({"United States":"US"}, inplace=True)
general_data.Country.replace({"Czech Republic":"Czechia"}, inplace=True)
general_data.Country.replace({"Taiwan":"Taiwan*"}, inplace=True)

# merge general data with epidemic data using SQL
from sqlalchemy import create_engine
db = create_engine('sqlite://', echo=False)
country_data.to_sql("country", con=db)
general_data.to_sql("general", con=db) 

query = """ 
SELECT *
FROM country c
INNER JOIN general g
ON c.countries=g.country
""" 
merge_data1 = pd.read_sql(query, con=db) 

#Drop useless columns and columns that do not seem relevant
merge_data1.drop(['Country','Region', 'index', 'Coastline (coast/area ratio)', 'Other (%)', 'Climate'],axis=1,inplace=True)
merge_data1.dtypes

#Convert columns to float
to_replace = ["Pop. Density (per sq. mi.)","Net migration","Infant mortality (per 1000 births)","Literacy (%)", 
              "Phones (per 1000)", "Arable (%)","Crops (%)","Birthrate","Deathrate", "Agriculture", "Industry", "Service"]
for i in to_replace:
  merge_data1[i] = merge_data1[i].str.replace(',', '.')

cols = merge_data1.columns.drop('Countries')
merge_data1[cols] = merge_data1[cols].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN (only 2 of them so no efficiency loss)
merge_data1 = merge_data1.dropna()

merge_data1

,Countries,Growth_rate,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Birthrate,Deathrate,Agriculture,Industry,Service
0,Algeria,0.187929,32930091,2381740,13.8,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,17.14,4.61,0.101,0.600,0.298
2,Argentina,0.199326,39921833,2766890,14.4,0.61,15.18,11200.0,97.1,220.4,12.31,0.48,16.73,7.55,0.095,0.358,0.547
3,Armenia,0.181127,2976372,29800,99.9,-6.47,23.28,3500.0,98.6,195.7,17.55,2.30,12.07,8.23,0.239,0.343,0.418
4,Australia,0.251654,20264082,7686850,2.6,3.98,4.69,29000.0,100.0,565.5,6.55,0.04,12.14,7.51,0.038,0.262,0.700
5,Austria,0.279480,8192880,83870,97.7,2.00,4.66,30000.0,98.0,452.2,16.91,0.86,8.74,9.76,0.018,0.304,0.678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Turkey,0.270246,70413958,780580,90.2,0.00,41.04,6700.0,86.5,269.5,30.93,3.31,16.62,5.97,0.117,0.298,0.585
68,Ukraine,0.179268,46710816,603700,77.4,-0.39,20.34,5400.0,99.7,259.9,56.21,1.61,8.82,14.39,0.187,0.452,0.361
69,United Arab Emirates,0.187621,2602713,82880,31.4,1.03,14.51,23200.0,77.9,475.3,0.60,2.25,18.96,4.40,0.040,0.585,0.375
70,United Kingdom,0.299958,60609153,244820,247.6,2.19,5.16,27700.0,99.0,543.5,23.46,0.21,10.71,10.13,0.005,0.237,0.758


## Step 2.2 World bank data


In [113]:
URL2 = "https://raw.githubusercontent.com/AlexOktay/covid19/master/API_3_DS2_en_csv_v2_896475.csv"
economics = pd.read_csv(URL2)

#Data cleaning
economics = economics.drop([0,1,2,3], axis=0)
economics = economics.pivot(index='Data Source', columns='Unnamed: 2', values='Unnamed: 4')
economics = economics.rename_axis('Country_Name')

rename_countries={'Congo, Dem. Rep.':'Congo (Kinshasa)', 'Congo, Rep.':'Congo (Brazzaville)', 'Korea, Rep.':'Korea, South',
                  'St. Kitts and Nevis':'Saint Kitts and Nevis', 'St. Lucia':'Saint Lucia',
                  'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines', 'United States':'US', 'Czech Republic':'Czechia',
                  'Egypt, Arab Rep.':'Egypt', 'Iran, Islamic Rep.':'Iran', 'Russian Federation':'Russia'}

economics = economics.rename(index=rename_countries)
merge_data1.to_sql("source", con=db)
economics.to_sql("econ", con=db)

query = """ 
SELECT *
FROM source s
INNER JOIN econ e
ON e.Country_name = s.countries
""" 
merge_data2 = pd.read_sql(query, con=db) 
#Drop useless columns and columns that do not seem relevant
merge_data2 = merge_data2[['Countries', 'Growth_rate', 'Population', 'Area (sq. mi.)', 'Pop. Density (per sq. mi.)', 'Net migration',
       'Infant mortality (per 1000 births)', 'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)','Arable (%)','Crops (%)','Birthrate','Deathrate',
       'Agriculture','Industry','Service', 'Adjusted net national income per capita (constant 2010 US$)','Current account balance (% of GDP)',
       'Exports of goods and services (% of GDP)','External debt stocks (% of GNI)','Foreign direct investment, net inflows (% of GDP)',
       'Foreign direct investment, net outflows (% of GDP)','Gross capital formation (% of GDP)','Gross domestic savings (% of GDP)',
       'Imports of goods and services (% of GDP)','Inflation, consumer prices (annual %)','Short-term debt (% of total reserves)','Total debt service (% of GNI)','Trade (% of GDP)']]
merge_data2

,Countries,Growth_rate,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Birthrate,Deathrate,Agriculture,Industry,Service,Adjusted net national income per capita (constant 2010 US$),Current account balance (% of GDP),Exports of goods and services (% of GDP),External debt stocks (% of GNI),"Foreign direct investment, net inflows (% of GDP)","Foreign direct investment, net outflows (% of GDP)",Gross capital formation (% of GDP),Gross domestic savings (% of GDP),Imports of goods and services (% of GDP),"Inflation, consumer prices (annual %)",Short-term debt (% of total reserves),Total debt service (% of GNI),Trade (% of GDP)
0,Algeria,0.187929,32930091,2381740,13.8,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,17.14,4.61,0.101,0.600,0.298,3651.397735,-13.178013,22.661204,3.459552,0.717464,-0.002017,48.047998,37.492329,33.217950,5.591116,1.999293,0.145520,55.879155
1,Argentina,0.199326,39921833,2766890,14.4,0.61,15.18,11200.0,97.1,220.4,12.31,0.48,16.73,7.55,0.095,0.358,0.547,9110.293202,-4.916457,11.242721,37.756505,1.791961,0.179810,18.757232,16.019342,13.980611,NaN,100.264613,6.374368,25.223332
2,Armenia,0.181127,2976372,29800,99.9,-6.47,23.28,3500.0,98.6,195.7,17.55,2.30,12.07,8.23,0.239,0.343,0.418,3131.939101,-2.988021,37.329503,86.138062,2.176847,0.252066,19.289686,7.651455,49.500638,0.969553,38.416431,12.112162,86.830141
3,Australia,0.251654,20264082,7686850,2.6,3.98,4.69,29000.0,100.0,565.5,6.55,0.04,12.14,7.51,0.038,0.262,0.700,42703.690700,-2.694728,21.193200,NaN,3.574156,0.626482,24.070736,24.687193,20.576800,1.948647,NaN,NaN,41.770000
4,Austria,0.279480,8192880,83870,97.7,2.00,4.66,30000.0,98.0,452.2,16.91,0.86,8.74,9.76,0.018,0.304,0.678,39821.386540,1.538314,54.037671,NaN,3.240018,2.422403,24.765517,28.291363,50.735505,2.081269,NaN,NaN,104.773176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Turkey,0.270246,70413958,780580,90.2,0.00,41.04,6700.0,86.5,269.5,30.93,3.31,16.62,5.97,0.117,0.298,0.585,11982.384620,-5.552749,24.773794,54.273805,1.354089,0.316767,30.993473,26.482751,29.284516,11.144311,110.751448,10.282454,54.058310
64,Ukraine,0.179268,46710816,603700,77.4,-0.39,20.34,5400.0,99.7,259.9,56.21,1.61,8.82,14.39,0.187,0.452,0.361,2964.827278,-2.176658,48.014298,105.731682,2.519824,0.208574,19.946968,12.257723,55.703543,14.438323,118.330553,11.984825,103.717841
65,United Arab Emirates,0.187621,2602713,82880,31.4,1.03,14.51,23200.0,77.9,475.3,0.60,2.25,18.96,4.40,0.040,0.585,0.375,35004.047100,NaN,101.679240,NaN,2.741380,2.741380,24.370987,49.062663,76.987564,1.966826,NaN,NaN,178.666804
66,United Kingdom,0.299958,60609153,244820,247.6,2.19,5.16,27700.0,99.0,543.5,23.46,0.21,10.71,10.13,0.005,0.237,0.758,36880.889900,-3.489236,30.366125,NaN,4.547794,5.177332,17.524631,16.321880,31.579013,2.557756,NaN,NaN,61.945139


# Step 3: Create and fit the model
Build our model using ML

In [114]:
#With XGBoost

#Preparation
data = merge_data2
y = data.Growth_rate
data_features = list(data.columns)
data_features.remove("Countries")
data_features.remove("Growth_rate")
X=data[data_features]

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split (X, y, train_size=0.99, test_size=0.01, random_state=5)
# With train size = 0.85, average mean absolute error is approx 0.03
# We set a train_size of 99% after validation to get the most accurate results in our final prediction

#Fit model and mpute missing data with median
from xgboost import XGBRegressor
from sklearn.impute import KNNImputer

my_imputer = KNNImputer(n_neighbors=2)
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

model = XGBRegressor(n_estimators=100, learning_rate=0.10)
model.fit(X_train, y_train, 
	        eval_set=[(X_valid, y_valid)],
          verbose=False)

# Uncomment for cross validation and prediction test using train size. Results: Mean_absolute_error = 0.31
"""
from sklearn.metrics import mean_absolute_error
predictions = model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
pipeline1 = Pipeline(steps=[('preprocessor', SimpleImputer(strategy='median')),
                              ('model', XGBRegressor(n_estimators=100, learning_rate = 0.10, random_state=0))
                             ])
scores = -1 * cross_val_score(pipeline1, X, y, cv=10, scoring='neg_mean_absolute_error')
print("Average MAE score (across experiments):")
print(scores.mean())
"""

[10:53:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


'\nfrom sklearn.metrics import mean_absolute_error\npredictions = model.predict(X_valid)\nprint("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))\n\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.pipeline import Pipeline\npipeline1 = Pipeline(steps=[(\'preprocessor\', SimpleImputer(strategy=\'median\')),\n                              (\'model\', XGBRegressor(n_estimators=100, learning_rate = 0.10, random_state=0))\n                             ])\nscores = -1 * cross_val_score(pipeline1, X, y, cv=10, scoring=\'neg_mean_absolute_error\')\nprint("Average MAE score (across experiments):")\nprint(scores.mean())\n'

# Step 4: predict data using model

## Step 4.1: data preparation

In [115]:
# Gather the list of countries we didn't use for training
for index, row in country_data_copy.iterrows():
  if country_data_copy.loc[index, last_update] > 250:
    country_data_copy = country_data_copy.drop(index)
  else:
    pass
country_data_copy.drop(country_data_copy.iloc[:, 0:], axis=1, inplace=True)
general_data.Country.replace({"Antigua & Barbuda":"Antigua and Barbuda"}, inplace=True)
general_data.Country.replace({"Bahamas, The":"Bahamas"}, inplace=True)
general_data.Country.replace({"Bosnia & Herzegovina":"Bosnia and Herzegovina"}, inplace=True)
general_data.Country.replace({"Cape Verde":"Cabo Verde"}, inplace=True)
general_data.Country.replace({"Central African Rep.":"Central African Republic"}, inplace=True)
general_data.Country.replace({"Congo, Repub. of the, The":"Congo (Brazzaville)"}, inplace=True)
general_data.Country.replace({"Congo, Dem. Rep, The":"Congo (Kinshasa)"}, inplace=True)
general_data.Country.replace({"Gambia, The":"Gambia"}, inplace=True)
general_data.Country.replace({"Macedonia":"North Macedonia"}, inplace=True)
general_data.Country.replace({"Saint Kitts & Nevis":"Saint Kitts and Nevis"}, inplace=True)
general_data.Country.replace({"East Timor":"Timor-Leste"}, inplace=True)
general_data.Country.replace({"Trinidad & Tobago":"Trinidad and Tobago"}, inplace=True)
general_data.Country.replace({"Gaza Strip":"West Bank and Gaza"}, inplace=True)
#Countries left out due to missing data: Eswatini, Holy See, Kosovo, Montenegro

# Join with required data, and do the data cleaning as done in step 2
country_data_copy.to_sql("pred", con=db)
general_data.to_sql("gen", con=db)

query = """ 
SELECT *
FROM pred c
INNER JOIN gen g
ON c.countries=g.country
INNER JOIN econ e
ON e.Country_name = c.countries
""" 
prediction_data = pd.read_sql(query, con=db) 
prediction_data.drop(['Country','Region', 'index', 'Coastline (coast/area ratio)', 'Other (%)', 'Climate'],axis=1,inplace=True)
prediction_data = prediction_data[['Countries', 'Population', 'Area (sq. mi.)', 'Pop. Density (per sq. mi.)', 'Net migration',
       'Infant mortality (per 1000 births)', 'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)','Arable (%)','Crops (%)','Birthrate','Deathrate',
       'Agriculture','Industry','Service', 'Adjusted net national income per capita (constant 2010 US$)','Current account balance (% of GDP)',
       'Exports of goods and services (% of GDP)','External debt stocks (% of GNI)','Foreign direct investment, net inflows (% of GDP)',
       'Foreign direct investment, net outflows (% of GDP)','Gross capital formation (% of GDP)','Gross domestic savings (% of GDP)',
       'Imports of goods and services (% of GDP)','Inflation, consumer prices (annual %)','Short-term debt (% of total reserves)','Total debt service (% of GNI)','Trade (% of GDP)']]

to_replace = ["Pop. Density (per sq. mi.)","Net migration","Infant mortality (per 1000 births)","Literacy (%)", 
              "Phones (per 1000)", "Arable (%)","Crops (%)","Birthrate","Deathrate", "Agriculture", "Industry", "Service"]
for i in to_replace:
  prediction_data[i] = prediction_data[i].str.replace(',', '.')

cols = prediction_data.columns.drop('Countries')
prediction_data[cols] = prediction_data[cols].apply(pd.to_numeric, errors='coerce')

country_list_pred = prediction_data["Countries"]
prediction_data.drop(['Countries'], axis=1, inplace=True)

prediction_data

,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Birthrate,Deathrate,Agriculture,Industry,Service,Adjusted net national income per capita (constant 2010 US$),Current account balance (% of GDP),Exports of goods and services (% of GDP),External debt stocks (% of GNI),"Foreign direct investment, net inflows (% of GDP)","Foreign direct investment, net outflows (% of GDP)",Gross capital formation (% of GDP),Gross domestic savings (% of GDP),Imports of goods and services (% of GDP),"Inflation, consumer prices (annual %)",Short-term debt (% of total reserves),Total debt service (% of GNI),Trade (% of GDP)
0,31056997,647500,48.0,23.06,163.07,700.0,36.0,3.2,12.13,0.22,46.60,20.34,0.380,0.240,0.380,NaN,-21.430835,5.904816,13.389900,0.255222,0.055769,19.174225,7.155705,45.332065,4.975952,5.366420,0.313625,51.236881
1,3581655,28748,124.6,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,15.11,5.22,0.232,0.188,0.579,4226.515631,-7.541137,31.556646,77.061501,7.852228,-0.815168,25.049155,8.791835,46.604666,1.986661,59.884852,3.924523,78.161312
2,12127071,1246700,9.7,0.00,191.19,1900.0,42.0,7.8,2.41,0.24,45.11,24.20,0.096,0.658,0.246,1650.887960,-0.518218,29.004100,44.445771,-6.057209,1.107077,24.130305,29.881683,23.252721,31.691686,21.168801,7.754783,52.256821
3,69108,443,156.0,-6.15,19.46,11000.0,89.0,549.9,18.18,4.55,16.93,5.37,0.038,0.220,0.743,NaN,-8.846589,NaN,NaN,10.690529,-0.159342,NaN,NaN,NaN,2.432488,NaN,NaN,NaN
4,7961619,86600,91.9,-4.90,81.74,3400.0,97.0,137.1,19.63,2.71,20.74,9.75,0.141,0.457,0.402,NaN,4.122197,48.547865,39.553139,7.016879,6.275164,24.378926,31.072197,41.854451,12.904891,10.099674,5.821852,90.402316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3431932,176220,19.5,-0.32,11.95,12800.0,98.0,291.4,7.43,0.23,13.91,9.05,0.093,0.311,0.596,12560.028310,0.741779,21.429093,NaN,4.695802,8.653766,15.190344,18.258250,18.334826,6.218094,NaN,NaN,39.763919
85,27307134,447400,61.0,-1.72,71.10,1700.0,99.3,62.9,10.83,0.83,26.36,7.84,0.342,0.229,0.430,NaN,2.501535,21.801105,27.934950,3.038105,0.015248,29.473116,26.929982,23.877797,NaN,2.985739,3.164667,45.678902
86,84402966,329560,256.1,-0.45,25.95,2500.0,90.3,187.7,19.97,5.95,16.86,6.22,0.209,0.410,0.381,1623.341693,-0.736885,101.593436,48.867452,6.300835,0.214497,26.582114,25.457998,98.791145,3.520257,44.621191,6.551421,200.384580
87,11502010,752614,15.3,0.00,88.29,800.0,80.6,8.2,7.08,0.03,41.00,19.93,0.220,0.290,0.489,NaN,-3.890374,35.151981,70.869858,3.347373,-0.796330,NaN,NaN,33.793711,6.577312,45.395833,3.408538,68.945692


## Step 4.2: prediction

In [116]:
#Make prediction
prediction_values = model.predict(prediction_data)
final_prediction = pd.DataFrame({"Countries": country_list_pred})
final_prediction["Estimated exp growth rate"]=prediction_values
final_prediction.set_index("Countries", inplace=True)
final_prediction.sort_values(by="Estimated exp growth rate", inplace=True, ascending=False)

#Uncomment to download results
#final_prediction.to_csv("results.csv", sep=",")
#from google.colab import files
#files.download('results.csv')

final_prediction

,Estimated exp growth rate
Countries,
Monaco,0.278225
Cuba,0.242953
Somalia,0.239873
San Marino,0.235365
Chad,0.232597
...,...
Mauritania,0.180151
Oman,0.180035
Guyana,0.178261
